In [59]:
import pandas as pd
import sqlite3
from thefuzz import fuzz
from thefuzz import process


# Load the data
conn = sqlite3.connect('../../data/raw/master.db')

odds_data = pd.read_csv('../../data/processed/odds.csv')

print(odds_data.head())


  Div        Date            HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  \
0  D1  24/08/2018       Bayern Munich     Hoffenheim     3     1   H     1   
1  D1  25/08/2018  Fortuna Dusseldorf       Augsburg     1     2   A     1   
2  D1  25/08/2018            Freiburg  Ein Frankfurt     0     2   A     0   
3  D1  25/08/2018              Hertha       Nurnberg     1     0   H     1   
4  D1  25/08/2018          M'gladbach     Leverkusen     2     0   H     0   

   HTAG HTR  ...  1XBCH  1XBCD  1XBCA  BFECH  BFECD  BFECA  BFEC>2.5  \
0     0   H  ...    NaN    NaN    NaN    NaN    NaN    NaN       NaN   
1     0   H  ...    NaN    NaN    NaN    NaN    NaN    NaN       NaN   
2     1   A  ...    NaN    NaN    NaN    NaN    NaN    NaN       NaN   
3     0   H  ...    NaN    NaN    NaN    NaN    NaN    NaN       NaN   
4     0   D  ...    NaN    NaN    NaN    NaN    NaN    NaN       NaN   

   BFEC<2.5  BFECAHH  BFECAHA  
0       NaN      NaN      NaN  
1       NaN      NaN      NaN  
2 

/var/folders/k5/sqpgl_zj3yg5kn6h04ymp5z40000gn/T/ipykernel_47108/637560664.py:10: DtypeWarning: Columns (130) have mixed types. Specify dtype option on import or set low_memory=False.
  odds_data = pd.read_csv('../../data/processed/odds.csv')


In [60]:
teams_query = """
SELECT distinct home_team FROM Match
"""
teams_df = pd.read_sql_query(teams_query, conn)

print(teams_df.head())


        home_team
0         Arsenal
1         Watford
2  Crystal Palace
3       West Brom
4         Chelsea


In [61]:
# Get unique teams from odds data
odds_home_teams = set(odds_data['HomeTeam'].unique())
odds_away_teams = set(odds_data['AwayTeam'].unique())
odds_teams = list(odds_home_teams.union(odds_away_teams))


# Create a mapping dictionary starting with exact matches
team_name_mapping = {}

# For each team that only exists in match data, find closest match in odds data
for odds_team in odds_teams:
    # Find closest matching team from odds data using process.extractOne
    # This returns tuple of (match, score, index)
    best_match = process.extractOne(odds_team, teams_df['home_team'], scorer=fuzz.ratio)
    
    if best_match[1] >= 40:  # Only include matches with 80% or higher similarity
        team_name_mapping[odds_team] = best_match[0]
        print(f"Fuzzy match: {odds_team} -> {best_match[0]} (Score: {best_match[1]})")

print("\nTotal mappings found:", len(team_name_mapping))
print("\nSample of mappings (including exact matches):")
for k, v in list(team_name_mapping.items())[:5]:
    print(f"{k} -> {v}")


Fuzzy match: Cadiz -> Cádiz (Score: 80)
Fuzzy match: Fortuna Dusseldorf -> Düsseldorf (Score: 64)
Fuzzy match: Clermont -> Clermont Foot (Score: 76)
Fuzzy match: West Brom -> West Brom (Score: 100)
Fuzzy match: Brighton -> Brighton (Score: 100)
Fuzzy match: Venezia -> Venezia (Score: 100)
Fuzzy match: Rennes -> Rennes (Score: 100)
Fuzzy match: Milan -> Milan (Score: 100)
Fuzzy match: Aston Villa -> Aston Villa (Score: 100)
Fuzzy match: Arsenal -> Arsenal (Score: 100)
Fuzzy match: Nimes -> Nîmes (Score: 80)
Fuzzy match: Inter -> Inter (Score: 100)
Fuzzy match: Troyes -> Troyes (Score: 100)
Fuzzy match: Tottenham -> Tottenham (Score: 100)
Fuzzy match: Dortmund -> Dortmund (Score: 100)
Fuzzy match: Lens -> Lens (Score: 100)
Fuzzy match: Heidenheim -> Heidenheim (Score: 100)
Fuzzy match: Benevento -> Benevento (Score: 100)
Fuzzy match: Stoke -> Stoke City (Score: 67)
Fuzzy match: Ein Frankfurt -> Eint Frankfurt (Score: 96)
Fuzzy match: Atalanta -> Atalanta (Score: 100)
Fuzzy match: Reims -

In [62]:
team_name_mapping["Leeds"] = "Leeds United"
team_name_mapping["Bielefeld"] = "Arminia"
team_name_mapping["Man City"] = "Manchester City"
team_name_mapping["Man United"] = "Manchester Utd"


In [63]:
# Convert mapping dictionary to DataFrame
mapping_df = pd.DataFrame.from_dict(team_name_mapping, orient='index', columns=['mapped_name']).rename(columns={'mapped_name': 'odds_name'})
mapping_df.index.name = 'match_name'

# Save to CSV
mapping_df.to_csv('../../data/processed/team_name_mapping.csv')

print("Mapping saved to data/processed/team_name_mapping.csv")
print(f"Total mappings saved: {len(mapping_df)}")

Mapping saved to data/processed/team_name_mapping.csv
Total mappings saved: 149


In [64]:
odds_data['HomeTeam'] = odds_data['HomeTeam'].apply(lambda x: team_name_mapping[x] if x in team_name_mapping else x)
odds_data['AwayTeam'] = odds_data['AwayTeam'].apply(lambda x: team_name_mapping[x] if x in team_name_mapping else x)

odds_data.to_csv('../../data/processed/odds.csv', index=False)